In [2]:
import requests
from bs4 import BeautifulSoup
import re
import os
import time

In [3]:
# URL
url = 'http://www.slmpd.org/CrimeReport.aspx'

# Path to save location
path = 'raw_data/'

In [4]:
def get_filename(headers):
    """Parses out the filename from a response header."""
    return headers['content-disposition'].split('=')[1]

###### Get all the dataset eventtargets

There five hidden form parameters on the page.<br><br>
\__EVENTARGUMENT<br>
\__EVENTVALIDATION<br>
\__VIEWSTATE<br>
\__EVENTTARGET<br>
\__VIEWSTATEGENERATOR<br>

The first page can be requested by a normal get request to the url.
The other page requests require 'Argument' and 'Target' arguments.

Target, Validation, State, and Generator are required for requests for the files. Some combination of them uniquely identify which file should be returned. The values for Target are reused, so at least one of those other parameters is necesary.

In [5]:
payload = {}

In [6]:
# The first page request is a get to the url.
r = requests.get(url)
soup = BeautifulSoup(r.content)

# Get the three hidden parameter values.
payload_raw = soup.find_all('input')
payload = {x['name']:x['value'] for x in payload_raw}

# List to hold eventtargets.
datasets_eventtargets_raw = []

# Get the data for this page and store it.
links = soup.find_all(href=re.compile("javascript:__doPostBack\('.*D',''\)"))
datasets_eventtargets_raw.append((1, dict(payload), links))
    
# Set EventTarget for page requesting.
payload['__EVENTTARGET'] = 'GridView1'

# Loop through all pages.
for i in xrange(2,7):
    # Set the eventargument value in the payload.
    payload['__EVENTARGUMENT'] = 'Page$' + str(i)
    
    # Request the page, make a soup object, get all relevant tags.
    r = requests.post(url, data=payload)
    soup = BeautifulSoup(r.content)
    # Get the three hidden parameter values.
    inputs_raw = soup.find_all('input')
    inputs = {x['name']:x['value'] for x in inputs_raw}
    links = soup.find_all(href=re.compile("javascript:__doPostBack\('.*D',''\)"))
    datasets_eventtargets_raw.append((i, inputs, links))

datasets_eventtargets_raw now contains a list of tuples
1. page number
2. dict of inputs for three of the five parameters to the js function
3. list of js function calls that contain the argument values for requesting the files

In [9]:
# Get list of files that have already been downloaded.
file_list = set(os.listdir('raw_data/'))

In [13]:
# Loop through the list of tuples and use the payload dict from each tuple to call all the files from the list in that tuple.
pat = re.compile(r"\(\'(.+?)\'\)?")
for tup in datasets_eventtargets_raw:
    # Parse out the argument value and filename (for validating responses).
    datasets_eventtargets = [(pat.findall(x['href'])[0], x.text) for x in tup[2]]
    
    # Get the three common arguments for all the files on this page.
    payload = tup[1]
    
    # Add a blank fourth.
    payload['__EVENTARGUMENT'] = ''
    
    # Loop through the parsed file arguments and request the files.
    for t in datasets_eventtargets:
        if t[1] not in file_list:  # Check if this file has already been downloaded.
            payload['__EVENTTARGET'] = t[0]
            r = requests.post(url, data=payload)
            if get_filename(r.headers) == t[1]:
                # Save the file. 
                # TODO: Should rename the files so year is first so they sort correctly.
                with open(os.path.join(path, get_filename(r.headers)), 'wb') as f:  
                    f.write(r.content)
            else:
                print('Error with page: ' + str(tup[0]) + ', argument: ' + t[0])
            time.sleep(5) #to avoid connection issues with the server
        else:
            print(t[1] + ' has already been downloaded.')

October2015.CSV has already been downloaded.
September2015.CSV has already been downloaded.
August2015.CSV has already been downloaded.
July2015.CSV has already been downloaded.
June2015.CSV has already been downloaded.
May2015.CSV has already been downloaded.
April2015.CSV has already been downloaded.
March2015.CSV has already been downloaded.
February2015.CSV has already been downloaded.
January2015.CSV has already been downloaded.
December2014.CSV has already been downloaded.
November2014.CSV has already been downloaded.
October2014.CSV has already been downloaded.
September2014.CSV has already been downloaded.
August2014.CSV has already been downloaded.
July2014.CSV has already been downloaded.
June2014.CSV has already been downloaded.
May2014.CSV has already been downloaded.
April2014.CSV has already been downloaded.
March2014.CSV has already been downloaded.
February2014.CSV has already been downloaded.
January2014.CSV has already been downloaded.
December2013.CSV has already bee